In [1]:
# From command line execute following command
# python makemytrip.py 

import urllib.request, urllib.error, urllib.parse
from urllib.request import urlretrieve
import json
import sys 
import re
#import datetime
from datetime import date, time, datetime
#from datetime import datetime, date, time
#import mapping as city
import pickle
from dateutil.rrule import rrule, DAILY
import tqdm
from functools import partial
from datetime import date, datetime, timedelta
import time
import multiprocessing
import os

BASE_URL="http://flights.makemytrip.com/makemytrip/"

class MakeMyTrip(object):
    
    def __init__(self):
        self.url_browse = ""
        self.flights_data = ""
        self.stoppage = ""
        self.arrival_time = ""
        self.trip_json = []
        
    def browse(self, url="", roundtrip=False):
        #This Prints the url being retrived. Not required for complete autonomous mode
        #print(url)
        try:
            self.url_browse = urllib.request.urlopen(url).read()
        except urllib.error.HTTPError:  
            print("There was an ERROR: Probably your internet connection \
                  isn't working")
            return 1
        urlretrieve(url, 'out.txt')
        i = 0
        fil = open("out.txt","r")
        if roundtrip:
            json_list = json.loads(fil.read())
            #print json_list#['filterData']
            print("-"*50)
            json_list = json.loads(json_list['fd'])
            return json_list
        for line in fil.readlines():
            i = i+1
            if "flightsData" in line:
                self.flights_data = line
                break
        temp_flights_data = self.flights_data.replace("var flightsData = ","").strip()
        temp_flights_data = temp_flights_data[:-1]
        fil = open("out.txt","w")
        fil.write(temp_flights_data)
        fil.close()
        try :
            json_list = json.loads(temp_flights_data)
        except ValueError : 
            json_list = 1
        return json_list
    
    def create_json_oneway(self, dump_list):
        for i in range(len(dump_list)):
            temp = '{ "airline" : "' + dump_list[i]['le'][0]['an'] + '"'
            temp = temp + ', "price" : "' + str(dump_list[i]['af']) + '"'
            temp = temp + ', "total_time" : "' + str(dump_list[i]['td']) + '"'
            temp = temp + ', "depart_date" : "' + str(dump_list[i]['le'][0]['fd']) + '"'
            temp = temp + ', "depart_time" : "' + str(dump_list[i]['le'][0]['fdt']) + '"'
            temp_dump_list = dump_list[i]['le']
            for x in range(len(temp_dump_list)):
                if x == (len(temp_dump_list)-1):
                    temp = temp + ', "arrival_date" : "' + str(temp_dump_list[x]['fa']) + '"'
                    temp = temp + ', "arrival_time" : "' + str(temp_dump_list[x]['fat']) + '"}'
            self.trip_json.append(temp)
        return json.dumps(self.trip_json)
        
    def create_json_roundtrip(self, dump_list):
        #Todo : Complete this function to return the custom JSON as response 
        for i in range(len(dump_list)):
            return json.loads(['fd'])
        
    def journey_oneway(self, origin, destination, depart_date, fare_class, adult=1, children=0, infant=0):
        adult = str(adult) if adult >= 1 else "1"
        children = str(children) if children >= 1 else str(children)
        infant = str(infant) if infant >= 1 else str(infant)
        new_url = BASE_URL + "search/O/O/"+fare_class+"/" + adult +"/" + children + "/" + infant + "/S/V0/" + origin + "_" + destination + "_" + depart_date
        return self.browse(new_url)
    
    def journey_roundtrip(self, origin, destination, depart_date, return_date, adult=1, children=0, infant=0):
        new_url = BASE_URL + 'splitRTDataService.json?classType=E&deptDate=' + depart_date + '&fltMap=&fromCity='+ origin + '&noOfAdlts=' + str(adult) + \
        '&noOfChd=' + str(children) + '&noOfInfnt=' + str(infant) + '&returnDate=' + return_date + '&toCity=' + destination + '&tripType=R&tripTypeDup=R'
        return self.browse(new_url, True)
        
    #Todo: Get rid of this method
    def read_line(self):
        flights_data=""
        i = 0
        fil = open("out.txt","r")
        for line in fil.readlines():
            i = i+1
            if "flightsData" in line:
                flights_data = line
        #print "Total lines",i
        self.format_flights_data(flights_data)
        #self.getFlightTable(flights_data)
        
    #Todo: Get rid of this method
    def format_flights_data(self, flights_data):
        new_flights_data = flights_data.replace("var flightsData = ","").strip()
        new_flights_data = new_flights_data[:-1]
        fil = open("out.txt","wb")
        fil.write(new_flights_data)
        fil.close()
        d = new_flights_data
        li = json.loads(d)
        self.create_json_oneway(li)
        #print type(new_flights_data)
            
    def get_extra_detail(self, flights_data):
        #date_size=len(flights_data)
        halt = flights_data[0]['f']
        layover = ""
        for x in range(len(flights_data)):
            halt = halt + "   \u2708   " + flights_data[x]['t'] + " ( " + flights_data[x]['du'] + " )"
            if x > 0:
                layover = layover + flights_data[x]['f'] + "  ( " + flights_data[x]['lo'] + " )  "
            if x == (len(flights_data)-1):
                self.arrival_time = flights_data[x]['fa'] + " " +flights_data[x]['fat']
        print(halt)
        return layover
        #return halt
        
    def print_json(self, l):
        
        tmp_size = len(l)
        for i in range(tmp_size):   
            print("")
            print("\033[1m" + l[i]['le'][0]['an'] , "\033[0m      \u20B9 \033[92m", l[i]['af'], "\033[0m  in  ", l[i]['td'])
            layover=self.get_extra_detail(l[i]['le']) 
            #ToDo
            print(l[i]['le'][0]['fd'],  l[i]['le'][0]['fdt'], \
             "  --->>  ", self.arrival_time)
            print("\tStoppage : ", layover)
            #print "Arrival : ", l[i]['le'][0]['fa'], l[i]['le'][0]['fat']
            print("\u2982"*50)

 
 #To write the required files into json format

    def file_json(self,l,destination, origin , flight_date):
        os.makedirs("data", exist_ok=True)
        os.makedirs("data/data_raw", exist_ok=True)
        with open(f"data/data_raw/price_data_retrived_{str(date.fromtimestamp(time.time()))}.csv", "a") as f1:
            if l == 1 :
                taken_date = str(datetime.today().date())
                f1.write("Origin" + "," + "Destination" + "," + "Dept_Date" + "," + "Dept_Time" + "," + "Arr_Time" + "," + "Total_Fare" + "," + "Base_Fare" + "," + "Fuel_Fare" + "," + "Airways" + "," + "Available" + "," + "Duration" + "," + "Class_Type" + "," + "Flight Number" + "," + "Flight Code" + "," + "FlightID" + "," + "Hopping" + "," +"Taken" +"\n")
                f1.write(origin + "," + destination + "," + flight_date + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + "," + "NA" + ","+ "NA" + "," + "NA" + "," + "NA" + "," + "NA" + ","+ taken_date +"\n")
            else :
                tmp_size = len(l)
                f1.write("Origin" + "," + "Destination" + "," + "Dept_Date" + "," + "Dept_Time" + "," + "Arr_Time" + "," + "Total_Fare" + "," + "Base_Fare" + "," + "Fuel_Fare" + "," + "Airways" + "," + "Available" + "," + "Duration" + "," + "Class_Type" + "," + "Flight Number"+ "," + "Flight Code" + "," + "FlightID" + "," + "Hopping" + "," +"Taken" +"\n")
                for i in range(tmp_size):  
                    airways = l[i]['le'][0]['an'] 
                    fare = l[i]['af']
                    deptdate = l[i]['le'][0]['dep']
                    depttime = l[i]['le'][0]['fdt']
                    arrtime = l[i]['le'][0]['fat']
                    avail = l[i]['le'][0]['flightFare']['bookingClass']['availability']
                    basefare = l[i]['le'][0]['flightFare']['baseFare']
                    fuel_surcharge = l[i]['le'][0]['flightFare']['fuelSurcharge']
                    duration = l[i]['td']
                    origin = l[i]['le'][0]['o']
                    desti = l[i]['le'][0]['d']
                    class_type = l[i]['le'][0]['cls']
                    flight_number = l[i]['le'][0]['fn']
                    flight_code = l[i]['le'][0]['oc']
                    Flight_ID = l[i]['fi']
                    hopping = l[i]['hff']

                    '''
                    if hopping == True :
                        arrtime = l[i]['le'][1]['fat']
                    else :
                        arrtime = l[i]['le'][0]['fat']
                    '''
                    taken_date = str(datetime.today().date())

                    f1.write(origin + "," + desti + "," + deptdate + "," +
                             depttime + "," + arrtime + "," + str(fare) + "," + str(basefare) +
                             "," + str(fuel_surcharge) + "," +airways+ "," + avail + ","+duration +
                             "," + class_type + ","+ flight_number + "," + flight_code + ","  +
                             Flight_ID + "," + str(hopping) + "," +taken_date+  "\n")

if __name__=="__main__":
    #Date as (yyyy, mm, dd)
    till_date = date(2019, 5, 16)
    time_period=till_date-date.today() #class datetime.timedelta
    if time_period.days<180: till_date=date.today()+timedelta(180)
        
    origin_dest_pairs=[("BLR", "MAA"), ("BOM", "DEL"), ("BOM", "JAI")]
    fare_class=       ["E", "B"]
    dept_dates=       [str(dt.strftime("%d-%m-%Y")) for dt in 
                            rrule(DAILY, dtstart=date.fromtimestamp(time.time()), until=till_date)]
    args_list=[(x,y,z) for x in origin_dest_pairs for y in fare_class for z in dept_dates]
    
    def download_data(args):
        origin, destination=args[0]
        fare_class=args[1]
        dept_date=args[2]
        bro = MakeMyTrip()
        bro.file_json(bro.journey_oneway(origin,destination,dept_date,fare_class), destination, origin , dept_date)
        
        return 1
    
    pool = multiprocessing.Pool(processes=49)  # Num of Parallel Processes
    _ = sum(tqdm.tqdm(pool.imap_unordered(download_data, args_list), total=len(args_list), leave=False))

  0%|          | 0/1086 [00:00<?, ?it/s]

There was an ERROR: Probably your internet connection                   isn't working


  4%|▍         | 48/1086 [00:39<06:17,  2.75it/s]  

There was an ERROR: Probably your internet connection                   isn't working


 14%|█▎        | 149/1086 [02:20<59:36,  3.82s/it]  

...

There was an ERROR: Probably your internet connection                   isn't working


100%|█████████▉| 1084/1086 [14:05<00:02,  1.12s/it]

There was an ERROR: Probably your internet connection                   isn't working


100%|█████████▉| 1085/1086 [14:07<00:01,  1.33s/it]

There was an ERROR: Probably your internet connection                   isn't working
